In [1]:
import os
from dotenv import load_dotenv


load_dotenv("DB_details.env")  

db_config = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT")),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASS"),
    "database": os.getenv("DB_NAME")

}


print("Loaded Database config successfully", db_config)

Loaded Database config successfully {'host': '127.0.0.1', 'port': 3306, 'user': 'root', 'password': '#Eduvos1121', 'database': 'video_store'}


In [5]:
import mysql.connector
from datetime import date

def movie_availibility(videoId, videoVer):
    try:
        conn = mysql.connector.connect(**db_config)
        cur = conn.cursor()
        cur.execute("""
            SELECT COUNT(*)
            FROM hire
            WHERE videoId = %s AND videoVer = %s AND dateReturn IS NULL
        """, (videoId, videoVer))
        count = cur.fetchone()[0]
        cur.close()
        conn.close()

        if count == 0:
            print("Video is available")
            return True
        else:
            print("Video is currently out")
            return False

    except Exception as err:
        print("Error (movie_availibility):", err)
        return False


def movie_hire(custId, videoId, videoVer):
    try:
        if not movie_availibility(videoId, videoVer):
            print("Movie is currently hired, Select a new movie")
            return False

        conn = mysql.connector.connect(**db_config)
        cur = conn.cursor()
        cur.execute("""
            INSERT INTO hire (custId, videoId, videoVer, dateHired, dateReturn)
            VALUES (%s, %s, %s, %s, NULL)
        """, (custId, videoId, videoVer, date.today()))
        conn.commit()
        cur.close()
        conn.close()

        print("Movie has been added to hire")
        return True

    except Exception as err:
        print("Error (movie_hire):", err)
        return False


def movie_returns(videoId, videoVer):
    try:
        conn = mysql.connector.connect(**db_config)
        cur = conn.cursor()
        cur.execute("""
            UPDATE hire
            SET dateReturn = %s
            WHERE videoId = %s AND videoVer = %s AND dateReturn IS NULL
        """, (date.today(), videoId, videoVer))
        conn.commit()
        rows = cur.rowcount
        cur.close()
        conn.close()

        if rows:
            print("Return has been successfully logged")
            return True
        else:
            print("No return available for that hire.")
            return False

    except Exception as err:
        print("Error (movie_returns):", err)
        return False
